**Santander 2025 - Ciência de Dados com Python**

**Desafio: Explorando IA Generativa em um Pipeline de ETL com Python**

In [23]:
import pandas as pd
import requests
import json
import os

**Extract**

In [16]:
sdw2025_api_url = "https://sdw-fake-api.onrender.com"

In [12]:
from google.colab import files
files.upload()

Saving SDW2025.csv to SDW2025.csv


{'SDW2025.csv': b'UserID\r\n1\r\n2\r\n3\r\n4\r\n5\r\n6\r\n7\r\n8\r\n9\r\n10'}

In [13]:
df = pd.read_csv('SDW2025.csv', sep=";")
user_ids = df['UserID'].tolist()
print(user_ids)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [17]:
def get_user(id):
  response = requests.get(f'{sdw2025_api_url}/users/{id}')
  return response.json() if response.status_code == 200 else None

users = [user for id in user_ids if (user := get_user(id)) is not None]
print(json.dumps(users, indent=2))

[
  {
    "id": 1,
    "name": "Naruto Uzumaki",
    "account": {
      "id": 1,
      "number": "00001-1",
      "agency": "0001",
      "balance": 1500,
      "limit": 500
    },
    "card": {
      "id": 1,
      "number": "**** **** **** 1111",
      "limit": 1000
    },
    "features": [],
    "news": []
  },
  {
    "id": 2,
    "name": "Hinata Hyuga",
    "account": {
      "id": 2,
      "number": "00002-2",
      "agency": "0001",
      "balance": 2300,
      "limit": 800
    },
    "card": {
      "id": 2,
      "number": "**** **** **** 2222",
      "limit": 1500
    },
    "features": [],
    "news": []
  },
  {
    "id": 3,
    "name": "Sasuke Uchiha",
    "account": {
      "id": 3,
      "number": "00003-3",
      "agency": "0001",
      "balance": 3200,
      "limit": 1200
    },
    "card": {
      "id": 3,
      "number": "**** **** **** 3333",
      "limit": 2000
    },
    "features": [],
    "news": []
  },
  {
    "id": 4,
    "name": "Sakura Haruno",
    "account

**Transform**

In [33]:
os.environ["GROQ_API_KEY"] = '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'

In [30]:
!pip -q install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 3.5 MB/s eta 0:00:00


In [34]:
from groq import Groq

In [35]:
groq = Groq(api_key=os.getenv("GROQ_API_KEY"))

def generate_ai_news(user):
    try:
        resp = groq.chat.completions.create(
            model="llama-3.1-8b-instant",
            messages=[
                {"role": "system", "content": "Você é um especialista em marketing bancário."},
                {"role": "user", "content": f"Crie uma mensagem para {user['name']} sobre a importância dos investimentos (máximo de 100 caracteres)."}
            ],
            temperature=0.7,
        )
        # garante <= 100 caracteres
        return resp.choices[0].message.content.strip()[:100]
    except Exception:
        # fallback caso estoure limite (sem quebrar o ETL)
        return f"{user['name']}, investir regularmente ajuda a construir um futuro financeiro mais seguro."

In [36]:
for user in users:
    news = generate_ai_news(user)
    print(news)
    user["news"].append({
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
        "description": news
    })

"Naruto, um ninho de riqueza é como um ninjutsu de segurança financeira! Investe agora e construa se
"Hinata, desenvolve suas habilidades financeiras como você desenvolve sua técnica de Senso! Investim
"Sasuke, não deixe sua força se esgotar em busca de vingança. Investe em sua estabilidade financeira
"Sakura, junte sua energia de ninja ao mercado de ações! Investir é um ataque poderoso para o seu fu
"Senhor Kakashi, seus jutsus não renderiam tanto se não investisse em seu futuro. Diversifique seus 
"Shikamaru, não perca tempo! Investir agora pode garantir um futuro seguro e confortável para você e
"Ino, não deixe seus sonhos se voarem! Investir em sua carreira e futuro é o primeiro passo para alc
"Choji, a força de um ninja não é apenas na batalha, mas também na construção de um futuro sólido co
"Rock Lee, seu futuro é forte! Investe em sua economia, como você investe em suas habilidades. Cresç
"Gaara, o controle da chama é apenas o começo! Aprenda a controlar seu dinheiro com investi

**Load**

In [38]:

def update_user(user):
  response = requests.put(f"{sdw2025_api_url}/users/{user['id']}", json=user)
  return True if response.status_code == 200 else False

for user in users:
  success = update_user(user)
  print(f"User {user['name']} updated? {success}!")

User Naruto Uzumaki updated? True!
User Hinata Hyuga updated? True!
User Sasuke Uchiha updated? True!
User Sakura Haruno updated? True!
User Kakashi Hatake updated? True!
User Shikamaru Nara updated? True!
User Ino Yamanaka updated? True!
User Choji Akimichi updated? True!
User Rock Lee updated? True!
User Gaara updated? True!
